In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
import random
import sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import random
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
import tensorflow as tf

In [4]:


# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
maxwell=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\maxwell_dataset.csv",header=None)
columns_maxwell=['Syear','App','Har','Dba','Ifc','Source','Telonuse','Nlan','T01','T02','T03','T04','T05','T06','T07','T08','T09','T10','T11','T12','T13','T14','T15','Duration','Size','Time','Effort']
maxwell.set_axis(columns_maxwell,axis='columns',inplace=True)
maxwell.set_axis(range(1,63),axis=0 ,inplace=True)
maxwell.rename_axis("Features", axis=1,inplace=True)
maxwell.rename_axis("Projects", axis=0,inplace=True)
dataset = maxwell

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort']).values  # Features
y = dataset['Effort'].values  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.function(reduce_retracing=True)

#=========================================================================================================================
# Define the fitness function
def fitness_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    cross_val_rmse = []
    
    X_selected = X[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
        # Step 3: Build the ANN model.
        model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

        # Step 4: Compile the model.
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Step 5: Train the model.
        model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=0)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
        
    return 1 / (1 + np.mean(error_list))  # Fitness is the inverse of the error (higher is better)
#========================================================================================================================

max_fitness_score=0
max_score_features=0

# Genetic Algorithm Parameters
population_size = 10
num_generations = 100
crossover_rate = 0.8
mutation_rate = 0.1

# Create an initial population of feature subsets
population = []
for _ in range(population_size):
    subset = [random.randint(0, 1) for _ in range(X.shape[1])]
    population.append(subset)
    
# Genetic Algorithm
for generation in range(num_generations):
    # Evaluate fitness for each feature subset in the population
    fitness_scores = [fitness_function(subset) for subset in population]
    if max(fitness_scores) > max_fitness_score:
        max_fitness_score = max(fitness_scores)
        max_score_features =population[fitness_scores.index(max(fitness_scores))]
    
    # Selection
    selected_population = sorted(population, key=lambda x: fitness_function(x), reverse=True)
    
    # Crossover
    elitism_num = 2
    offspring_population = selected_population[:elitism_num] # Elitism, preserving the top individuals
    for i in range(0, population_size, 2):
        parent1 = selected_population[i]
        parent2 = selected_population[i + 1]
        
        if random.random() < crossover_rate:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = parent1[:crossover_point] + parent2[crossover_point:]
            child2 = parent2[:crossover_point] + parent1[crossover_point:]
        else:
            child1 = parent1
            child2 = parent2
        
        offspring_population.append(child1)
        offspring_population.append(child2)
    
    # Mutation
    for i in range(2,population_size):
        if random.random() < mutation_rate:
            mutated_feature = random.randint(0, X.shape[1] - 1)
            offspring_population[i][mutated_feature] = 1 - offspring_population[i][mutated_feature]
    
    # Replace the old population with the new offspring population
    population = offspring_population

# Select the best feature subset from the final population
best_subset = max(population, key=fitness_function)
selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), best_subset) if is_selected]

print(selected_features)
#======================================================================================================================    
    
# Define the ANN model to be optimized.
def ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=int(neurons_input), activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(int(num_hidden_layers)):
        model.add(tf.keras.layers.Dense(units=int(neurons_hidden), activation='relu'))

    model.add(tf.keras.layers.Dense(units=1))  # Output layer with a single unit for regression.

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Define the search space for Bayesian optimization.
pbounds = {
        'neurons_input': (10, 50),
        'neurons_hidden': (10, 50),
        'num_hidden_layers': (1, 5),
        'learning_rate': (1e-5, 1e-2),
        'batch_size': (8, 32),
        'epochs': (5,20)
}

# Define the function to optimize (minimize RMSE).
def optimize_effort_estimation(neurons_input, neurons_hidden, num_hidden_layers, learning_rate, batch_size, epochs):
    model = ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate)
    
    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(epochs), verbose=0)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=optimize_effort_estimation,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )

    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
        
    #build the model
    model = ann_model(best_params['neurons_input'],best_params['neurons_hidden'],best_params['num_hidden_layers'],best_params['learning_rate'])
    # Train the model.
    model.fit(X_train, y_train, epochs=int(best_params['epochs']), batch_size=int(best_params['batch_size']), verbose=0)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)


    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
                
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")               
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")            
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")
#===================================================================================================================

# train the model using max_features

# X_selected = X[:, max_score_features]
# X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# model3 = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)),
#     tf.keras.layers.Dense(units=32, activation='relu'),
#     tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
# ])

#  # Step 4: Compile the model.
# model3.compile(optimizer='adam', loss='mean_squared_error')

#  # Step 5: Train the model.
# model3.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=1)

# # You can now use the trained model for prediction or further evaluation
# y_pred = model3.predict(X_test_scaled)

# # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
# rmse3 = np.sqrt(mean_squared_error(y_test, y_pred))
# print(f"Root Mean Squared Error (RMSE): {rmse3}")

# # Convert y_pred and y_test to NumPy arrays
# y_pred = np.array(y_pred).flatten()
# y_test = np.array(y_test)

# # Calculate Mean Magnitude of Relative Error (MMRE)
# mmre = np.mean(np.abs((y_test - y_pred) / y_test))
# print(f"Mean Magnitude of Relative Error (MMRE): {mmre:.2f}")




# print(best_subset," \n ", max_score_features)




1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 40ms/step


...

1/1 [==============================] - 0s 43ms/step
| 2         | -4.54e+03 | 9.394     | 17.99     | 0.006015  | 38.32     | 10.82     | 4.88      |
1/1 [==============================] - 0s 39ms/step
| 3         | -1.36e+04 | 27.98     | 8.185     | 0.001826  | 17.34     | 22.17     | 3.099     |
1/1 [==============================] - 0s 38ms/step
| 4         | -1.359e+0 | 18.37     | 9.368     | 0.006122  | 15.58     | 21.69     | 2.465     |
1/1 [==============================] - 1s 911ms/step
| 5         | -1.359e+0 | 18.95     | 16.78     | 0.002005  | 30.57     | 33.7      | 1.186     |
1/1 [==============================] - 0s 44ms/step
| 6         | -1.36e+04 | 22.58     | 7.558     | 0.0006599 | 47.96     | 48.63     | 4.234     |
1/1 [==============================] - 0s 35ms/step
| 7         | -1.356e+0 | 15.31     | 6.465     | 0.006845  | 27.61     | 14.88     | 2.981     |
1/1 [==============================] - 0s 41ms/step
| 8         | -6.186e+0 | 8.825     | 18.64    

1/1 [==============================] - 0s 48ms/step
| 40        | -4.474e+0 | 8.0       | 14.49     | 0.01      | 24.13     | 40.08     | 5.0       |
1/1 [==============================] - 0s 47ms/step
|   iter    |  target   | batch_... |  epochs   | learni... | neuron... | neuron... | num_hi... |
-------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 36ms/step
| 1         | -4.635e+0 | 16.99     | 19.26     | 0.007323  | 33.95     | 16.24     | 1.624     |
1/1 [==============================] - 0s 45ms/step
| 2         | -3.338e+0 | 9.394     | 17.99     | 0.006015  | 38.32     | 10.82     | 4.88      |
1/1 [==============================] - 0s 40ms/step
| 3         | -4.757e+0 | 27.98     | 8.185     | 0.001826  | 17.34     | 22.17     | 3.099     |
1/1 [==============================] - 0s 37ms/step
| 4         | -4.741e+0 | 18.37     | 9.368     | 0.006122  | 15.58     | 21.69     | 2.465     |
1/

1/1 [==============================] - 0s 40ms/step
| 37        | -4.755e+0 | 27.8      | 6.016     | 0.009573  | 23.59     | 11.92     | 3.756     |
1/1 [==============================] - 0s 33ms/step
| 38        | -4.757e+0 | 22.29     | 5.297     | 0.0003438 | 18.54     | 20.36     | 1.734     |
1/1 [==============================] - 0s 34ms/step
| 39        | -4.611e+0 | 8.781     | 7.507     | 0.00901   | 45.43     | 40.68     | 1.908     |
1/1 [==============================] - 0s 44ms/step
| 40        | -4.722e+0 | 13.39     | 16.41     | 0.002722  | 11.39     | 16.17     | 4.896     |
1/1 [==============================] - 0s 43ms/step
|   iter    |  target   | batch_... |  epochs   | learni... | neuron... | neuron... | num_hi... |
-------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 34ms/step
| 1         | -6.938e+0 | 16.99     | 19.26     | 0.007323  | 33.95     | 16.24     | 1.624     |
1/

1/1 [==============================] - 0s 39ms/step
| 34        | -5.481e+0 | 9.572     | 9.772     | 0.004336  | 49.95     | 12.57     | 3.785     |
1/1 [==============================] - 0s 39ms/step
| 35        | -7.131e+0 | 20.82     | 12.09     | 0.0002202 | 33.14     | 35.18     | 3.528     |
1/1 [==============================] - 0s 42ms/step
| 36        | -6.311e+0 | 23.37     | 17.94     | 0.004866  | 23.08     | 49.27     | 3.769     |
1/1 [==============================] - 0s 40ms/step
| 37        | -7.129e+0 | 27.8      | 6.016     | 0.009573  | 23.59     | 11.92     | 3.756     |
1/1 [==============================] - 0s 34ms/step
| 38        | -7.13e+03 | 22.29     | 5.297     | 0.0003438 | 18.54     | 20.36     | 1.734     |
1/1 [==============================] - 0s 38ms/step
| 39        | -5.492e+0 | 9.356     | 9.681     | 0.00653   | 36.96     | 32.45     | 2.2       |
1/1 [==============================] - 0s 37ms/step
| 40        | -6.761e+0 | 25.79     | 18.14     

1/1 [==============================] - 0s 48ms/step
| 31        | -1.398e+0 | 32.0      | 5.0       | 0.01      | 50.0      | 10.0      | 5.0       |
1/1 [==============================] - 0s 48ms/step
| 32        | -1.415e+0 | 12.62     | 20.0      | 1e-05     | 19.12     | 44.19     | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 33        | -3.817e+0 | 8.0       | 17.26     | 0.01      | 13.74     | 50.0      | 5.0       |
1/1 [==============================] - 0s 34ms/step
| 34        | -1.407e+0 | 8.744     | 18.25     | 0.002538  | 17.06     | 48.04     | 1.039     |
1/1 [==============================] - 0s 47ms/step
| 35        | -3.664e+0 | 9.714     | 20.0      | 0.01      | 11.81     | 47.91     | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 36        | -3.071e+0 | 8.0       | 19.43     | 0.01      | 12.76     | 41.06     | 5.0       |
1/1 [==============================] - 0s 48ms/step
| 37        | -4.083e+0 | 8.0       | 15.66     